In [ ]:
# ============================================
# 0. Install dependencies
# ============================================
!pip install -q roboflow diffusers transformers accelerate safetensors pillow matplotlib numpy

# ============================================
# 1. Imports & Google Drive mount
# ============================================
from roboflow import Roboflow
import os, torch, random, shutil
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers.utils import logging
from torch import autocast
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from google.colab import drive

drive.mount('/content/drive')
logging.set_verbosity_error()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rf = Roboflow(api_key="7TI8mHPqsyInAEUtwLsa")
project = rf.workspace("en-zbzva").project("all-pepper-datasets-q5opc")
version = project.version(1)
dataset = version.download("voc")

dataset_dir = dataset.location

train_dir = os.path.join(dataset_dir, "train")
val_dir   = os.path.join(dataset_dir, "valid")
test_dir  = os.path.join(dataset_dir, "test")

train_imgs = [f for f in os.listdir(train_dir) if f.endswith(".jpg")]
val_imgs   = [f for f in os.listdir(val_dir)   if f.endswith(".jpg")]
test_imgs  = [f for f in os.listdir(test_dir)  if f.endswith(".jpg")]

print("Train images:", len(train_imgs))
print("Valid images:", len(val_imgs))
print("Test images:", len(test_imgs))



loading Roboflow workspace...
loading Roboflow project...
Train images: 1489
Valid images: 427
Test images: 211


In [ ]:
# # ============================================
# # 2. Download Roboflow dataset
# # ============================================
# rf = Roboflow(api_key="7TI8mHPqsyInAEUtwLsa")
# project = rf.workspace("en-zbzva").project("all-pepper-datasets-q5opc")
# version = project.version(1)
# dataset = version.download("yolov9")

# dataset_dir = dataset.location
# train_imgs = os.path.join(dataset_dir, "train/images")
# val_imgs   = os.path.join(dataset_dir, "valid/images")
# test_imgs  = os.path.join(dataset_dir, "test/images")

# train_lbl = os.path.join(dataset_dir, "train/labels")
# val_lbl   = os.path.join(dataset_dir, "valid/labels")
# test_lbl  = os.path.join(dataset_dir, "test/labels")

# print("Train images:", len(os.listdir(train_imgs)))
# print("Valid images:", len(os.listdir(val_imgs)))
# print("Test images:",  len(os.listdir(test_imgs)))


loading Roboflow workspace...
loading Roboflow project...
Train images: 1489
Valid images: 427
Test images: 211


In [ ]:
# ============================================
# 3. Classical pre-processing
#    (light tweaks ONLY, does NOT distort shapes)
# ============================================

def weak_light_prep(img):
    img = ImageEnhance.Brightness(img).enhance(0.65)
    img = ImageEnhance.Contrast(img).enhance(0.8)
    img = ImageEnhance.Color(img).enhance(0.85)
    return img

def foggy_prep(img):
    img = ImageEnhance.Brightness(img).enhance(0.95)
    img = ImageEnhance.Contrast(img).enhance(0.8)
    img = ImageEnhance.Color(img).enhance(0.9)
    return img

def fog_overlay(img, intensity=0.4):
    """Soft fog overlay that preserves ALL geometry."""
    w, h = img.size
    fog = Image.new("RGBA", img.size, (255,255,255,0))
    px = fog.load()
    for i in range(w):
        for j in range(h):
            alpha = int(255 * intensity * np.exp(-(j/h)))
            px[i, j] = (255,255,255,alpha)
    fog = fog.filter(ImageFilter.GaussianBlur(6))
    return Image.alpha_composite(img.convert("RGBA"), fog).convert("RGB")

In [ ]:
# ============================================
# 4. Stable Diffusion (safe settings)
# ============================================
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype="float16"
).to("cuda")

pipe.set_progress_bar_config(disable=True)

# VERY IMPORTANT: low strength = shape preserved!
STRENGTH_WEAK  = 0.20
STRENGTH_FOGGY = 0.22

GUIDANCE = 2.0
STEPS = 10

PROMPT_WEAK  = "same pepper image, cloudy weak soft light, natural color, minimal alteration"
PROMPT_FOGGY = "same peppers image, light fog, diffused soft light, natural geometry"

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# ============================================
# 5. Output dataset roots in Google Drive
# ============================================
root_weak  = "/content/drive/MyDrive/pepper_weaklight"
root_foggy = "/content/drive/MyDrive/pepper_foggy"

for root in [root_weak, root_foggy]:
    for split in ["train","valid","test_roboflow"]:
        os.makedirs(os.path.join(root, split, "images"), exist_ok=True)
        os.makedirs(os.path.join(root, split, "labels"), exist_ok=True)

print("Saving datasets to:")
print(root_weak)
print(root_foggy)

Saving datasets to:
/content/drive/MyDrive/pepper_weaklight
/content/drive/MyDrive/pepper_foggy


In [ ]:
# ============================================
# 6. Diffusion-safe augmentation (geometry preserved)
# ============================================
def generate_diffusion_safe(img_path, out_path, mode):
    """
    Uses extremely low strength to keep shapes & labels valid.
    """
    img = Image.open(img_path).convert("RGB")  # keep original resolution

    # PREP (no geometry change)
    if mode == "weak":
        base = weak_light_prep(img)
        prompt = PROMPT_WEAK
        strength = STRENGTH_WEAK
    else:
        base = foggy_prep(img)
        prompt = PROMPT_FOGGY
        strength = STRENGTH_FOGGY

    # --- Diffusion with shape preservation ---
    with autocast("cuda"):
        out = pipe(
            prompt=prompt,
            image=base,
            strength=strength,
            guidance_scale=GUIDANCE,
            num_inference_steps=STEPS
        ).images[0]

    # Light fog overlay (geometry preserved)
    if mode == "foggy":
        out = fog_overlay(out, intensity=0.20)

    out.save(out_path, quality=95)


In [ ]:
# ============================================
# 7. Split processing with SKIP logic
# ============================================
def process_split(src_img_dir, dst_root, mode, split_name):
    dst_img_dir = os.path.join(dst_root, split_name, "images")

    src_files = sorted([f for f in os.listdir(src_img_dir)
                        if f.lower().endswith((".jpg",".png",".jpeg"))])

    dst_files = sorted([f for f in os.listdir(dst_img_dir)
                        if f.lower().endswith((".jpg",".png",".jpeg"))])

    if len(dst_files) >= len(src_files):
        print(f"[{mode}] {split_name}: already exists ({len(dst_files)}/{len(src_files)}) → SKIP")
        return

    print(f"[{mode}] {split_name}: generating {len(src_files)} images...")

    for fname in src_files:
        inp = os.path.join(src_img_dir, fname)
        out = os.path.join(dst_img_dir, fname)
        if not os.path.exists(out):
            generate_diffusion_safe(inp, out, mode)

In [ ]:
# ============================================
# 8. Build both datasets
# ============================================

# Weak-Light
process_split(train_imgs, root_weak, "weak", "train")
process_split(val_imgs,   root_weak, "weak", "valid")
process_split(test_imgs,  root_weak, "weak", "test_roboflow")

# Foggy
process_split(train_imgs, root_foggy, "foggy", "train")
process_split(val_imgs,   root_foggy, "foggy", "valid")
process_split(test_imgs,  root_foggy, "foggy", "test_roboflow")

print("Diffusion-safe weak/foggy datasets generated!")

[weak] train: generating 1489 images...


KeyboardInterrupt: 

In [ ]:
# ============================================
# 9. Copy labels (labels are valid because geometry preserved!)
# ============================================
def copy_labels(src, dst_root, split):
    dst = os.path.join(dst_root, split, "labels")
    src_files = [f for f in os.listdir(src) if f.endswith(".txt")]
    dst_files = [f for f in os.listdir(dst) if f.endswith(".txt")]

    if len(dst_files) >= len(src_files):
        print(f"[LABELS] {dst_root}/{split}: SKIP")
        return

    print(f"[LABELS] {dst_root}/{split}: copying...")
    for f in src_files:
        shutil.copy(os.path.join(src, f), os.path.join(dst, f))

# Weak
copy_labels(train_lbl, root_weak, "train")
copy_labels(val_lbl,   root_weak, "valid")
copy_labels(test_lbl,  root_weak, "test_roboflow")

# Foggy
copy_labels(train_lbl, root_foggy, "train")
copy_labels(val_lbl,   root_foggy, "valid")
copy_labels(test_lbl,  root_foggy, "test_roboflow")

print("All labels copied")

In [ ]:
# ================================
# DataLoaders
# ================================
transform = transforms.Compose([
    transforms.ToTensor()
])

class PepperDataset(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir
        self.files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith((".jpg",".png",".jpeg"))])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        img = Image.open(os.path.join(self.img_dir, fname)).convert("RGB")
        return transform(img), fname

def make_loaders(root, batch=16):
    return (
        DataLoader(PepperDataset(os.path.join(root,"train","images")), batch_size=batch, shuffle=True),
        DataLoader(PepperDataset(os.path.join(root,"valid","images")), batch_size=batch, shuffle=False),
        DataLoader(PepperDataset(os.path.join(root,"test","images")),  batch_size=batch, shuffle=False),
    )

foggy_train_loader, foggy_val_loader, foggy_test_loader = make_loaders(root_foggy)
weak_train_loader,  weak_val_loader,  weak_test_loader  = make_loaders(root_weak)


print("All dataloaders ready!")
